In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp drive/MyDrive/IR_Project/cwishareddataset/traindevset/english/News_Train.tsv .
! cp drive/MyDrive/IR_Project/cwishareddataset/testset/english/News_Test.tsv .
! cp -r drive/MyDrive/IR_Project/cwishareddataset/traindevset/ .
! cp -r drive/MyDrive/IR_Project/cwishareddataset/testset/ .

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
word_label_dict = {}
seen_dict = {}

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
text=("Gus is running helping and caring organise a developed conference on Applications of Natural language processing. He keeps organising local Python meetups and several internal talks at his workplace.")
conference_help_doc = nlp(text)
# print(conference_help_doc)
for token in conference_help_doc:
    if str(token) != str(token.lemma_):
        print(f"{str(token):>20} : {str(token.lemma_)}")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


                  is : be
             running : run
             helping : help
              caring : care
           developed : develop
             Natural : natural
                  He : he
               keeps : keep
          organising : organise
             meetups : meetup
               talks : talk


In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
def process_input(text):
  text = text.lower()
  sen_tok_list = []
  wpt = WordPunctTokenizer()
  sen_tok = wpt.tokenize(text)
  sen_tok_list.append(sen_tok)
  final_word_list = []
  wnlem = WordNetLemmatizer()
  for i in range(len(sen_tok_list)):
    for j in range(len(sen_tok_list[i])):
      sen_tok_list[i][j] = wnlem.lemmatize(sen_tok_list[i][j])
  for i in range(len(sen_tok_list)):
    temp_word_list = []
    for j in range(len(sen_tok_list[i])):
      if sen_tok_list[i][j].isalpha() == True and sen_tok_list[i][j].isnumeric() == False:
        temp_word_list.append(sen_tok_list[i][j])
    final_word_list.append(temp_word_list)
  return final_word_list

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
def generate_final_word_list(file_path):
  dataframe = pd.read_csv(file_path,sep = '\t',header = None,names = ["ID","Sentence","complex_phrase_start_index","complex_phrase_end_index","Complex_phrase","num_native","num_n_native","native_complex","non-native complex","Binary values","probability values"])
  dataframe["Sentence"] = dataframe["Sentence"].apply(lambda x : x.lower())
  dataframe["Complex_phrase"] = dataframe["Complex_phrase"].apply(lambda x : x.lower())
  dataframe["Num_words"] = dataframe["Complex_phrase"].apply(lambda x : len(x.split()))
  dataframe = dataframe[dataframe["Num_words"] == 1]
  sentences_list = np.array(dataframe["Sentence"])
  labels = np.array(dataframe["Binary values"])
  complex_words = np.array(dataframe["Complex_phrase"])
  complex_word_dict = {complex_words[i]: labels[i] for i in range(len(labels))}
  sen_tok_list = []
  wpt = WordPunctTokenizer()
  for sentence in sentences_list:
    sen_tok = wpt.tokenize(sentence)
    sen_tok_list.append(sen_tok)
  wnlem = WordNetLemmatizer()
  for i in range(len(sen_tok_list)):
    for j in range(len(sen_tok_list[i])):
      sen_tok_list[i][j] = wnlem.lemmatize(sen_tok_list[i][j])
  final_word_list = []
  for i in range(len(sen_tok_list)):
    temp_word_list = []
    for j in range(len(sen_tok_list[i])):
      if sen_tok_list[i][j].isalpha() == True and sen_tok_list[i][j].isnumeric() == False:
        temp_word_list.append(sen_tok_list[i][j])
    final_word_list.append(temp_word_list)
  for sentence in final_word_list:
    for word in sentence:
      if word not in seen_dict:
        seen_dict[word] = 1
        if word not in complex_word_dict:
          word_label_dict[word] = 0
        else:
          word_label_dict[word] = complex_word_dict[word]

In [ ]:
generate_final_word_list("News_Train.tsv")
generate_final_word_list("traindevset/english/WikiNews_Train.tsv")
generate_final_word_list("traindevset/english/Wikipedia_Train.tsv")

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
import string
wnlem = WordNetLemmatizer()
def complex_word_check(word):
  if(word in string.punctuation):
    return False
  word = wnlem.lemmatize(word)
  if word in word_label_dict:
    return word_label_dict[word]
  else:
    return False

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install wordfreq
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=2b6982461c52c5e23968794fe9aa11989603efd46e5fa8332f3db90d649662f1
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

In [ ]:
import torch
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')


All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
def get_top_k_predictions(input_string, k=5, tokenizer=tokenizer, model=model) -> str:

    tokenized_inputs = tokenizer(input_string, return_tensors="tf")
    outputs = model(tokenized_inputs["input_ids"])

    top_k_indices = tf.math.top_k(outputs.logits, k).indices[0].numpy()
    decoded_output = tokenizer.batch_decode(top_k_indices)
    mask_token = tokenizer.encode(tokenizer.mask_token)[1:-1]
    mask_index = np.where(tokenized_inputs['input_ids'].numpy()[0]==mask_token)[0][0]

    decoded_output_words = decoded_output[mask_index]

    return decoded_output_words   

In [ ]:
from wordfreq import zipf_frequency
def sentence_complexity(sentence):
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(sentence)
  complexity_sum = 0
  num_tokens = len(tokens)
  for i in range(num_tokens):
    complexity_sum+= zipf_frequency(tokens[i], 'en')
  return complexity_sum

In [ ]:
from sentence_transformers import SentenceTransformer
bert_similarity_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
from wordfreq import zipf_frequency
from sklearn.metrics.pairwise import cosine_similarity
import textstat
def simplify_sentence(text):
  text = text.lower()
  wpt = WordPunctTokenizer()
  tokens = wpt.tokenize(text)
  complex_words = []
  index_of_complex_word = []
  for i in range(len(tokens)):
    if(textstat.difficult_words(tokens[i]) == 1):
      complex_words.append(tokens[i])
      index_of_complex_word.append(i)
  sentence1_transformed = bert_similarity_model.encode(text)
  for j in range(len(complex_words)):
    text_with_mask = text.replace(complex_words[j], '[MASK]')
    predictedWords = get_top_k_predictions(input_string = text_with_mask)
    predictedWords = predictedWords.split()
    word = ''
    similarity = []
    for i in range(len(predictedWords)):
      if(complex_words[j] == predictedWords[i]):
        continue
      simpler_text = text.replace(complex_words[j], predictedWords[i])
      sentence2_transformed = bert_similarity_model.encode(simpler_text)
      similarity.append(cosine_similarity([sentence1_transformed],[sentence2_transformed])[0][0])
    if(len(similarity) == 0):
      tokens[index_of_complex_word[j]] = complex_words[j]
    else:
      tokens[index_of_complex_word[j]] = predictedWords[similarity.index(max(similarity))]
  sentence = ""
  for token in tokens:
    sentence += token + " "
  return sentence

In [ ]:
from nltk.tokenize import WordPunctTokenizer
text2 = "Draconian laws made everybody miserable."
text3 = 'The door to his residence was locked'
text4 = 'Agree to my demands or i will intimidate you.'

simplified_text2 = simplify_sentence(text2)
simplified_text3 = simplify_sentence(text3)
simplified_text4 = simplify_sentence(text4)

print("Original text is: {} \nSimplified text is: {}".format(text4,simplified_text4))
print("Simplicity of original text is: {} \nSimplicity of simplified text is : {}".format(sentence_complexity(text4),sentence_complexity(simplified_text4)))
print("Original text is: {} \nSimplified text is: {}".format(text3,simplified_text3))
print("Simplicity of original text is: {} \nSimplicity of simplified text is : {}".format(sentence_complexity(text3),sentence_complexity(simplified_text3)))
print("Original text is: {} \nSimplified text is: {}".format(text2,simplified_text2))
print("Simplicity of original text is: {} \nSimplicity of simplified text is : {}".format(sentence_complexity(text2),sentence_complexity(simplified_text2)))

Original text is: Agree to my demands or i will intimidate you. 
Simplified text is: agree to my deal or i will torture you . 
Simplicity of original text is: 53.75 
Simplicity of simplified text is : 55.5
Original text is: The door to his residence was locked 
Simplified text is: the door to his apartment was locked 
Simplicity of original text is: 42.36 
Simplicity of simplified text is : 42.65
Original text is: Draconian laws made everybody miserable. 
Simplified text is: the laws made everybody suffer . 
Simplicity of original text is: 22.6 
Simplicity of simplified text is : 27.83


In [ ]:
input_text = ""
simplified_text = simplify_sentence(input_text)
print("Original text is: {} \nSimplified text is: {}".format(input_text,simplified_text))
print("Simplicity of original text is: {} \nSimplicity of simplified text is : {}".format(sentence_complexity(text4),sentence_complexity(simplified_text4)))

In [ ]:
# from wordfreq import zipf_frequency
# def simplify_sentence(text):
#   text = text.lower()
#   wpt = WordPunctTokenizer()
#   tokens = wpt.tokenize(text)
#   complex_words = []
#   index_of_complex_word = []
#   for i in range(len(tokens)):
#     if(complex_word_check(tokens[i]) == True):
#       complex_words.append(tokens[i])
#       index_of_complex_word.append(i)
#   for j in range(len(complex_words)):
#     encoded_text = tokenizer.encode(text, add_special_tokens=True)
#     complex_word_index = encoded_text.index(tokenizer.convert_tokens_to_ids(complex_words[j]))
#     encoded_text[complex_word_index] = tokenizer.mask_token_id
#     text_with_mask = tokenizer.decode(encoded_text)
#     predictedWords = get_top_k_predictions(input_string = text_with_mask)
#     predictedWords = predictedWords.split()
#     # print(predictedWords)
#     max = -1000000
#     word = ''
#     for i in range(len(predictedWords)):
#       temp = zipf_frequency(predictedWords[i], 'en') * (len(predictedWords) - i/2)
#       if temp>max:
#         max = temp
#         word = predictedWords[i]
#     tokens[index_of_complex_word[j]] = word
#   sentence = ""
#   for token in tokens:
#     sentence += token + " "
#   return sentence

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
